In [2]:
import torch
from torch import nn

class MLP(nn.Module):
    #声明带有模型参数的层，这里声明了两个全连接层
    def __init__(self, **kwargs):
        # 调用MLP父类Block的构造函数来进行必要的初始化。这样在构造实例时还可以指定其他函数
        super(MLP, self).__init__(**kwargs)
        self.hidden = nn.Linear(784, 256)
        self.act = nn.ReLU()
        self.output = nn.Linear(256, 10)
        
    # 定义模型的前向计算，即如何根据输入x计算返回所需要的模型输出
    def forward(self, x):
        o = self.act(self.hidden(x))
        return self.output(o)
        
        

**以上的 MLP 类中⽆须定义反向传播函数。系统将通过⾃动求梯度⽽自动⽣成反向传播所需的 backward 函数。**
我们可以实例化 MLP 类得到模型变量 net 。下⾯的代码初始化 net 并传入输⼊数据 X 做一次前向计算。其中， net(X) 会调用 MLP **继承自 Module 类的 __call__ 函数**，这个函数将调⽤用 MLP 类定义的forward 函数来完成前向计算。

In [3]:
X = torch.rand(2, 784)
net = MLP()
print(net)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


In [4]:
net(X)

tensor([[ 0.1442,  0.0418,  0.0844, -0.1390, -0.2477,  0.0780, -0.0380,  0.1658,
          0.0950,  0.1681],
        [ 0.2690,  0.0787,  0.1163, -0.1810, -0.2572,  0.0595, -0.0413,  0.0304,
         -0.0462,  0.1841]], grad_fn=<AddmmBackward0>)

### 自定义模型层

- 不含模型参数的层
    - 下⾯构造的 MyLayer 类通过继承 Module 类自定义了一个**将输入减掉均值后输出**的层，

In [5]:
import torch
from torch import nn

class MyLayer(nn.Module):
    def __init__(self, **kwargs):
        super(MyLayer, self).__init__(**kwargs)
    def forward(self, x):
        return x - x.mean()

In [6]:
layer = MyLayer()
print(layer)

MyLayer()


In [7]:
layer(torch.tensor([1,2,3,4,5], dtype=torch.float))

tensor([-2., -1.,  0.,  1.,  2.])

#### 含模型参数的层
我们还可以自定义含模型参数的自定义层。**其中的模型参数可以通过训练学出。**

- Parameter类其实是Tensor的子类,如果一个Tensor是Parameter，那么它会自动被添加到模型的参数列表里。
    - 所以在自定义含模型参数的层时，我们应该将参数定义成Parameter
    - 除了直接定义成Parameter类外，还可以使用ParameterList和parameterDict分别定义参数的列表和字典。

##### 两种定义模型参数的方式

In [12]:
class MyListDense(nn.Module):
    def __init__(self):
        super(MyListDense, self).__init__()
        self.params = nn.ParameterList([nn.Parameter(torch.randn(4, 4)) for i in range(3)])
        self.params.append(nn.Parameter(torch.randn(4, 1)))
        
    def forward(self, x):
        for i in range(len(self.params)):
            x = torch.mm(x, self.params[i])
        return x
    
net = MyListDense()
print(net)

MyListDense(
  (params): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 4x4]
      (1): Parameter containing: [torch.FloatTensor of size 4x4]
      (2): Parameter containing: [torch.FloatTensor of size 4x4]
      (3): Parameter containing: [torch.FloatTensor of size 4x1]
  )
)


In [14]:
class MyDictDense(nn.Module):
    def __init__(self):
        super(MyDictDense, self).__init__()
        self.params = nn.ParameterDict({
                'Linear1':nn.Parameter(torch.randn(4,4)),
                'Linear2':nn.Parameter(torch.randn(4,1))
        })
        self.params.update({'Linear3': nn.Parameter(torch.randn(4, 2))})  #新增
    
    def forward(self, x, choice='Linear1'):
        return torch.mm(x, self.params[choice])
    

net = MyDictDense()
print(net)

MyDictDense(
  (params): ParameterDict(
      (Linear1): Parameter containing: [torch.FloatTensor of size 4x4]
      (Linear2): Parameter containing: [torch.FloatTensor of size 4x1]
      (Linear3): Parameter containing: [torch.FloatTensor of size 4x2]
  )
)


#### 定义一些常见的卷积、池化层
#### 二维卷积层
- 二维卷积层将输入和卷积核做互相关运算，并加上一个标量偏差来得到输出。
    - 卷积层的模型参数包括了**卷积核**和**标量偏差**。
    - 在训练模型的时候，通常我们先对卷积核随机初始化，然后不断迭代卷积核和偏差。

In [17]:
import torch
from torch import nn

# 卷积运算（二维互相关）
def corr2d(X, K):
    h, w = K.shape
    X, K = X.float(), K.float()
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i + h, j: j + w] * K).sum()
    return Y

# 二维卷积层
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super(Conv2D, self).__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.randn(1))
        
    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [26]:
net = Conv2D((3,3))
print(net)

Conv2D()


- 填充(padding)是指在输⼊高和宽的两侧填充元素(通常是0元素)。

    - 下面的例子里我们创建一个⾼和宽为3的二维卷积层，然后设输⼊高和宽两侧的填充数分别为1。
    - 给定一 个高和宽为8的输入，我们发现输出的高和宽也是8。

In [27]:
import torch
from torch import nn

# 定义一个函数来计算卷积层。它对输入和输出做相应的升维和降维
def comp_conv2d(conv2d, X):
    #(1, 1)代表批量大小和通道数
    X = X.view((1, 1) + X.shape)    # view函数的意思是numpy中的reshape
    Y = conv2d(X)
    return Y.view(Y.shape[2:]) # 排除不关心的前两维：批量和通道

# 注意这里是两侧分别填充1行或列，所以在两侧一共填充2行或列
# 这里的Conv2d是pytorch集成的，而非上面自定义的Conv2d
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, padding=1)
X = torch.rand(8,8)
comp_conv2d(conv2d, X).shape

torch.Size([8, 8])